In [117]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [118]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%
#
# The traits should also be the file names, i.e. if the face file is:
# `./Chibishinobis09052021/1-face/f1.png`,
#
# then you should initialize the face traits array as:
# face = ["f1"]

attributes = {
    "1-face": {
        "files": ["f1"],
        "weights": [100]
    },
    "2-eyes": {
        "files": ["e1", "e2", "e3", "e4"],
        "weights": [7, 15, 15, 63]
    },
    "3-headbands": {
        "files": ["h1", "h2", "h3", "h4", "h5"],
        "weights": [5,15,18,22,40]
    },
    "4-innerheadband": {
        "files": ["i1"],
        "weights": [100]
    },
    "5-symbol": {
        "files": ["s1", "s2", "s3"],
        "weights": [57,29,14]
    },
    "6-cheek": {
        "files": ["c1", "c2", "c3", "c4", "c5"],
        "weights": [8,30,30,16,16]
    },
    "7-hair": {
        "files": ["hr1", "hr2", "hr3"],
        "weights": [15,60,25]
    }
}

In [119]:
## Sanity check calculation on the total number of possible combos
def num_possible_combinations(all_attrs):
    total_combos = 1
    for attribute in all_attrs:
        files = all_attrs[attribute]["files"]
        total_combos *= len(files)
    return total_combos

print(num_possible_combinations(attributes))

900


In [120]:
## Generate Traits

# Number of random unique images we want to generate
TOTAL_IMAGES = 200

all_images = [] 

# A recursive function to generate unique image combinations.
# Keep track of trait counts as we generate.
def create_new_image():
    
    new_image = {}
    
    # For each trait category, select a random trait based on the weightings 
    for key in attributes:
        new_image[key] = random.choices(attributes[key]["files"], attributes[key]["weights"])[0]
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    new_trait_image = create_new_image()
    all_images.append(create_new_image())
    
print("generated", len(all_images), "images!")

generated 200 images!


In [121]:
# Returns true if all images are unique.
# This is just a sanity check.. the create_new_image function should be generating unique images already.
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [122]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [123]:
for img in all_images:
    print(str(img) + "\n")

{'1-face': 'f1', '2-eyes': 'e4', '3-headbands': 'h5', '4-innerheadband': 'i1', '5-symbol': 's2', '6-cheek': 'c5', '7-hair': 'hr3', 'tokenId': 0}

{'1-face': 'f1', '2-eyes': 'e2', '3-headbands': 'h4', '4-innerheadband': 'i1', '5-symbol': 's1', '6-cheek': 'c3', '7-hair': 'hr2', 'tokenId': 1}

{'1-face': 'f1', '2-eyes': 'e2', '3-headbands': 'h5', '4-innerheadband': 'i1', '5-symbol': 's2', '6-cheek': 'c3', '7-hair': 'hr3', 'tokenId': 2}

{'1-face': 'f1', '2-eyes': 'e4', '3-headbands': 'h2', '4-innerheadband': 'i1', '5-symbol': 's2', '6-cheek': 'c2', '7-hair': 'hr2', 'tokenId': 3}

{'1-face': 'f1', '2-eyes': 'e4', '3-headbands': 'h4', '4-innerheadband': 'i1', '5-symbol': 's1', '6-cheek': 'c2', '7-hair': 'hr2', 'tokenId': 4}

{'1-face': 'f1', '2-eyes': 'e4', '3-headbands': 'h4', '4-innerheadband': 'i1', '5-symbol': 's3', '6-cheek': 'c2', '7-hair': 'hr3', 'tokenId': 5}

{'1-face': 'f1', '2-eyes': 'e3', '3-headbands': 'h4', '4-innerheadband': 'i1', '5-symbol': 's1', '6-cheek': 'c3', '7-hair': 

In [124]:
from collections import Counter

counters = {}
for att in attributes:
    counters[att] = Counter()

for img in all_images:
    for att in attributes:
        counters[att][img[att]] += 1
        
for att in counters:
    print(att,"weights ...")
    counter = counters[att]
    total_weight = sum(counter.values())
    for file in attributes[att]["files"]:
        actual_weight = counter[file] / total_weight
        weight_index = attributes[att]["files"].index(file)
        expected_weight = attributes[att]["weights"][weight_index]
        print(file, ":", "actual -", int(actual_weight*100),"expected -",expected_weight)
    print("")

1-face weights ...
f1 : actual - 100 expected - 100

2-eyes weights ...
e1 : actual - 9 expected - 7
e2 : actual - 21 expected - 15
e3 : actual - 19 expected - 15
e4 : actual - 50 expected - 63

3-headbands weights ...
h1 : actual - 5 expected - 5
h2 : actual - 21 expected - 15
h3 : actual - 20 expected - 18
h4 : actual - 20 expected - 22
h5 : actual - 33 expected - 40

4-innerheadband weights ...
i1 : actual - 100 expected - 100

5-symbol weights ...
s1 : actual - 48 expected - 57
s2 : actual - 32 expected - 29
s3 : actual - 19 expected - 14

6-cheek weights ...
c1 : actual - 10 expected - 8
c2 : actual - 27 expected - 30
c3 : actual - 27 expected - 30
c4 : actual - 17 expected - 16
c5 : actual - 18 expected - 16

7-hair weights ...
hr1 : actual - 18 expected - 15
hr2 : actual - 53 expected - 60
hr3 : actual - 28 expected - 25



In [125]:
from functools import reduce
    
#### Generate Images    
for image in all_images:
    #print("generating image: ", image["tokenId"])
    #print("loading layers...")
    layers = []
    for dirname, filename in image.items():
        if dirname == "tokenId":
            continue
        fullpath = f'./attributes/{dirname}/{filename}.png'
        opened = Image.open(fullpath).convert('RGBA')
        #print("opened layer at path:", fullpath)
        layers.append(opened)
    
    #print("composing image..")
    #print("adding layers 1 and 2")
    composite = Image.alpha_composite(layers[0], layers[1])
    for i in range(2, len(layers)):
        #print("adding layer", i+1)
        composite = Image.alpha_composite(composite, layers[i])
        
    rgb_im = composite.convert('RGB')
    file_name = str(image["tokenId"]+1) + ".png"
    rgb_im.save("./images/" + file_name)
    print("generated: ", file_name)
    

generated:  1.png
generated:  2.png
generated:  3.png
generated:  4.png
generated:  5.png
generated:  6.png
generated:  7.png
generated:  8.png
generated:  9.png
generated:  10.png
generated:  11.png
generated:  12.png
generated:  13.png
generated:  14.png
generated:  15.png
generated:  16.png
generated:  17.png
generated:  18.png
generated:  19.png
generated:  20.png
generated:  21.png
generated:  22.png
generated:  23.png
generated:  24.png
generated:  25.png
generated:  26.png
generated:  27.png
generated:  28.png
generated:  29.png
generated:  30.png
generated:  31.png
generated:  32.png
generated:  33.png
generated:  34.png
generated:  35.png
generated:  36.png
generated:  37.png
generated:  38.png
generated:  39.png
generated:  40.png
generated:  41.png
generated:  42.png
generated:  43.png
generated:  44.png
generated:  45.png
generated:  46.png
generated:  47.png
generated:  48.png
generated:  49.png
generated:  50.png
generated:  51.png
generated:  52.png
generated:  53.png
ge

In [91]:
#### Write the Metadata for all Traits out to a file
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [19]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Circle", i["Circle"]))
    token["attributes"].append(getAttribute("Square", i["Square"]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()